In [1]:
import pso
import dataset
import network
import numpy as np
import funcs

# One element for every weight + 2 at end for activation functions
num_values = (
    dataset.num_features * 4 # Input layer weights
    + 4 ** 2                 # Hidden layer weights
    + 4                      # Output layer weights
    + 2                      # Activation functions
    + 2 * 4 + 1              # Bias weights for all layers
)

min_values = -np.ones(num_values)
max_values = np.ones(num_values)

# 4 possible activation functions (0-1, 1-2, 2-3, 3-4)
min_values[-2:] = 0
max_values[-2:] = 3.999999999

swarm = pso.swarm(min_values, max_values)

model = network.network.from_list(swarm.search(), dataset.num_features)

In [2]:
y_predictions = model.forward_propagate(dataset.x)

print(np.mean(np.around(y_predictions) == dataset.y))

0.5553935860058309


In [9]:
# find average loss for each particle in the swarm
def swarm_fitness(x):
    n_particles = x.shape[0]
    loss = np.zeros(n_particles)
    for i in range(n_particles):
        net = network.network.from_list(x[i], dataset.num_features)            
        y_pred = net.forward_propagate(dataset.x)
        loss[i] = np.mean(funcs._log_loss(dataset.y, y_pred))

    return loss

In [19]:
import pyswarms as ps
# Initialize swarm
options = {'c1': 5.6, 'c2': 2.1, 'w':0.6}
bounds = (min_values, max_values)

# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=num_values, options=options, bounds=bounds)

# Perform optimization
cost, pos = optimizer.optimize(swarm_fitness, iters=1000, verbose=3)

2021-11-06 21:18:26,140 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 5.6, 'c2': 2.1, 'w': 0.6}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.687
2021-11-06 21:19:14,416 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.687100718881144, best pos: [ 0.04732996  0.83100987 -0.8157307   0.2219215   0.97179045  0.19271484
 -0.1673662  -0.0668689   0.08485388 -0.08827724 -0.56253465  0.96291148
  0.45051289  0.19531236  0.05412238  0.22622698  0.1323664   0.56501206
  0.29094525  0.89445799 -0.49338208 -0.96343911 -0.9704035  -0.326208
 -0.61096941 -0.53617431  0.32184765 -0.19649684 -0.44162633 -0.73814163
 -0.93344696  0.28390726 -0.41457881 -0.44198468 -0.24575348 -0.58973269
  0.0727745   0.68184245 -0.14255513 -0.61748015  0.25001938 -0.1584358
  0.18911911 -0.35957574 -0.14892563  0.93233204  0.0473717 ]


In [20]:
net = network.network.from_list(pos, dataset.num_features)            
y_pred = net.forward_propagate(dataset.x)
print(np.mean(np.around(y_predictions) == dataset.y))

0.5553935860058309
